In [2]:
import os
import csv
import glob
import shutil
import imghdr

In [3]:
# New Find Python File

def FindPythonFiles(filepath):
    
    spreadsheet_info = []                                           # Old path, first object number, second object number
    old_doc_nums = []                                               # Old image names (path and file name)
    object_nums = []                                                # New object numbers for each photo
    rename_docs = []                                                # Photos that need to be renamed
    old_nums = []                                                   # Old file name
    old_doc_nums_path = []                                          # Second half of path (portion after folder)
    file_type = []                                                  # Type of file
    doc_nums_dict = {}


    if filepath.endswith ('xlsx'):                                  # If is excel file
        excel_file = xlrd.open_workbook(filepath)
        sheet = excel_file.sheet_by_index(0)
        sheet.cell_value(0, 0)
        for i in range(sheet.nrows):
            spreadsheet_info.append(sheet.row_values(i))            # Makes list of each row in spreadsheet

    if filepath.endswith ('csv'):                                   # If is csv file
        with open(filepath) as csv_file:
            next(csv_file)
            csv_reader = csv.reader(csv_file, delimiter=',')
            for line in csv_reader:
                spreadsheet_info.append(line)                       # Makes list of each row in spreadsheet

    for i in spreadsheet_info:
        if i[1] != '':                                              # If image has corresponding object
            old_doc_nums.append(i[0])
            object_nums.append(i[1])
            old_nums.append((i[0].rsplit('\\', 1))[1])
            old_doc_nums_path.append('\\'.join(i[0].split('\\', 3)[3:]))
            file_type.append((i[0].rsplit('.', 1))[1])
        if i[2] != '':
            old_doc_nums.append(i[0])
            object_nums.append(i[2])
            old_nums.append((i[0].rsplit('\\', 1))[1])
            old_doc_nums_path.append('\\'.join(i[0].split('\\', 3)[3:]))
            file_type.append((i[0].rsplit('.', 1))[1])
    for a in range (len(object_nums)):
        if object_nums[a] not in doc_nums_dict:
            doc_nums_dict[object_nums[a]] = [old_doc_nums[a]]        # Dictionary of object number : original photo number
        else:
            doc_nums_dict[object_nums[a]].append(old_doc_nums[a])    # If object already in dictionary, adds to tuple

    folder = '\\'.join(filepath.split('\\', 3)[:3]) + '\\'           # One level above file/path column for spreadsheet
    #for root, dirs, files in os.walk(folder):                       # Check all the files and subfiles in the root folder
    #    for i in files:
    #        fullName = (os.path.join(root, i))
    #        if fullName in photo_nums_dict.values():                # See if each photo path matches an object
    #            rename_photos.append(fullName)                      # If photo matches object, run photo through renamer
    #            rename_photos.sort()

    return (doc_nums_dict, object_nums, folder, old_nums, old_doc_nums_path, old_doc_nums, file_type)

In [4]:
# Object ID List

from collections import defaultdict
from csv import DictReader

def ObjectIdList (key):
    filename = "objectlist.csv"
    d = {}

    with open(filename, 'r', encoding = 'utf-8') as data:
        next(data)
        reader = csv.reader(data)
        
        for line in reader:
            d[line[1] ] = int(line[0])                     # Object ID : Object Number

        if key in d:
            return(d[key])

In [5]:
# New Renamer

def remove(string): 
    return string.strip() 

def Renamer(filepath, dest_file):
    doc_nums_dict, object_nums, folder, old_nums, old_doc_nums_path, old_doc_nums, file_type = FindPythonFiles(filepath)
    filepath2 = filepath.rsplit('\\', 1)[0] + '\\'
    dest_file1 = dest_file + ".csv"
    write_file = os.path.join(filepath2, dest_file1)   # Change to new name of file
    filepath = filepath.strip()
    sheet_lines = []
    
    for i in range(len(old_nums)):                     # For every photo...
        line = ['']*38
        line[1] = ObjectIdList(object_nums[i])         # Second row is Object ID
        line[2] = object_nums[i]                       # Third row is Object Number
        line[3] = old_doc_nums_path[i]                 # Fourth row is File Name (second half of path)
        line[4] = folder                               # Fifth row is the path to the folder
        line[22] = 'document'                          # Media type
        line[23] = file_type[i]                        # Media format
        line[34] = False                               # Public access
        line[35] = '(not assigned)'
        line[36] = '(not assigned)'
        line[37] = False                               # Is primary display media record
        
        sheet_lines.append(line)

    with open(write_file, "w", newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')                  # Append to file, create if not exist
        file_header = ""
        header = ['ID', 'ObjectID', 'ObjectNumber', 'FileName', 'Path', 'PixelW', 'PixelH', 'IsColor', 'ColorDepthNumberOfBits', 'FileSize', 'MemorySize', 'FileDate', 'ThumbFileName', 'ThumbnailPath', 'RenditionNumber', 'RenditionDate', 'MediaDepartment', 'Photographer', 'Scanby', 'Technique', 'QuantityMade', 'CopyrightInfo', 'MediaType', 'MediaFormat', 'MediaView', 'Rank', 'ArchiveIDNumber', 'ArchCatNum', 'ArchVolName', 'ArchSubDir', 'ArchFilename', 'Description', 'PublicCaption', 'Remarks', 'PublicAccess', 'SemanticType', 'MediaStatus', 'IsPrimaryDisplayMediaRecord']
        for i in header:  
            file_header += i + ", "
        csv_file.write((file_header)+'\n')                            # Appends stripped line to list
        
        writer.writerows(sheet_lines)

In [6]:
filepath = r"Y:\TMS Media\....csv"     # Name of File (Columns = Image Path, Object Name, Object Name 2)

dest_file = input("Name of New File")
Renamer(filepath, dest_file)

Name of New Filetestg
